In [2]:
import pandas as pd
import ast
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/all_pages.csv')

# コラム厳選
columns_to_keep = [
    'job_offer_id',
    'job_offer_name',
    'client',
    'job_offer_areas',
    'job_offer_min_salary',
    'job_offer_max_salary',
    'job_offer_skill_names'
]

df_filtered = df.loc[:, columns_to_keep].copy()

def to_dict_if_needed(value):
    if value is None:
        return None
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None


def to_list_if_needed(value):
    if value is None:
        return None
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None

#データの整形
if 'client' in df_filtered.columns:
    df_filtered['client_dict'] = df_filtered['client'].apply(to_dict_if_needed)
    df_filtered['client_name'] = df_filtered['client_dict'].apply(lambda d: d.get('name') if isinstance(d, dict) else None)
    df_filtered['employee_count'] = df_filtered['client_dict'].apply(lambda d: d.get('employee_count') if isinstance(d, dict) else None)
    df_filtered['established'] = df_filtered['client_dict'].apply(lambda d: d.get('established_at') if isinstance(d, dict) else None)
    # df_filtered['established'] = df_filtered.to_datetime('established', unit='s')
    df_filtered['established'] = pd.to_datetime(df_filtered['established'], unit='s')
    df_filtered = df_filtered.drop(['client', 'client_dict'], axis=1, errors='ignore')

if 'job_offer_skill_names' in df_filtered.columns:
    df_filtered['job_offer_skill_names'] = df_filtered['job_offer_skill_names'].apply(to_list_if_needed)

if 'job_offer_areas' in df_filtered.columns:
    df_filtered['job_offer_areas'] = df_filtered['job_offer_skill_names'].apply(to_list_if_needed)

if 'job_offer_name' in df_filtered.columns:
    df_filtered['job_tag'] = 'その他'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア'), 'job_tag'] = 'データエンジニア'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア'), 'job_tag'] = 'AIエンジニア'

In [3]:
df_filtered.head()

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
0,294644,AIエンジニア【マネジメント・テックリード・新規事業リーダーへキャリアアップも可能】,"[JavaScript, Python, Go, Django, Node.js, Reac...",400,1000,"[JavaScript, Python, Go, Django, Node.js, Reac...",イグニション・ポイントフォース 株式会社,70,2020-06-30 15:00:00,AIエンジニア
1,294449,AIエージェント開発エンジニア,"[Java, Python, Go, Node.js, AWS, Docker, Azure...",500,1200,"[Java, Python, Go, Node.js, AWS, Docker, Azure...",ブーストコンサルティング 株式会社,20,2025-03-31 15:00:00,その他
2,294052,最先端AI技術を習得！未来を創る生成AIエンジニアを募集！,"[Java, Python]",500,1040,"[Java, Python]",10X 株式会社,20,2020-01-31 15:00:00,AIエンジニア
3,294554,AIエンジニア（PM候補）,"[AWS, Azure, GCP]",800,1200,"[AWS, Azure, GCP]",株式会社 Ridge-i,84,NaT,AIエンジニア
4,294524,【AI推進室】ソフトウェアエンジニア,[],750,1200,[],株式会社 カオナビ,394,2008-04-30 15:00:00,その他


In [4]:
df_filtered['job_tag']

0      AIエンジニア
1          その他
2      AIエンジニア
3      AIエンジニア
4          その他
        ...   
434        その他
435        その他
436        その他
437        その他
438    AIエンジニア
Name: job_tag, Length: 439, dtype: object

In [5]:
df_filtered[df_filtered['job_tag']=='データエンジニア']

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
7,294081,データエンジニア（メンバーポジション）,"[SQL, Python, AWS, Azure, GCP]",400,550,"[SQL, Python, AWS, Azure, GCP]",株式会社 cococo,10,2023-11-30 15:00:00,データエンジニア
12,187679,（職種未経験者歓迎）データエンジニア,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",400,600,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",株式会社 メンバーズ メンバーズデータアドベンチャーカンパニー,2303,1995-05-31 15:00:00,データエンジニア
21,273394,データエンジニア リーダー候補,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",600,1000,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",株式会社 FLINTERS,90,2013-12-31 15:00:00,データエンジニア
23,286281,データエンジニア（データ基盤構築）／リードエンジニア,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",500,650,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",株式会社 エスタイル,63,2006-06-30 15:00:00,データエンジニア
24,267012,データエンジニア,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",600,900,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",株式会社 ヘッドウォータース,290,2005-10-31 15:00:00,データエンジニア
...,...,...,...,...,...,...,...,...,...,...
403,272445,データエンジニア,[],480,800,[],株式会社 識学,227,2015-02-28 15:00:00,データエンジニア
404,211742,データエンジニア（MX統括部）,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",500,1000,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",パーソルビジネスプロセスデザイン 株式会社,18253,1977-08-31 15:00:00,データエンジニア
412,229877,データ基盤エンジニア,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",700,1000,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",アソビュー 株式会社,200,2011-02-28 15:00:00,データエンジニア
421,207594,データベースエンジニア,[SQL],400,600,[SQL],フレックシステムズ 株式会社,56,2003-09-30 15:00:00,データエンジニア


In [6]:
df['job_offer_areas'] = df['job_offer_areas'].apply(ast.literal_eval)
type(df.loc[2, "job_offer_skill_names"])

str

In [7]:
import pandas as pd

# エポック秒のSeries（10桁または13桁の数値）
epoch_series = pd.Series([1401548400, 1580482800, 1209567600])

# pd.to_datetime()で変換
# unit='s' は、入力が秒であることを示します
datetime_series = pd.to_datetime(epoch_series, unit='s')

print(datetime_series)

0   2014-05-31 15:00:00
1   2020-01-31 15:00:00
2   2008-04-30 15:00:00
dtype: datetime64[ns]


In [20]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv('../data/filtered.csv')

df_copy = df.copy()

df_copy['job_tag'] = 'その他'  # まず'その他'で初期化

df_copy.loc[df_copy['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア'), 'job_tag'] = 'データエンジニア'

df_copy.loc[df_copy['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'

df_copy.loc[df_copy['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア'), 'job_tag'] = 'AIエンジニア'


df_copy.loc[df['job_offer_min_salary']== 0 ,['job_offer_min_salary']].count()


median_salary = df_copy['job_offer_max_salary'].median()
df_copy['job_offer_max_salary'] = df_copy['job_offer_max_salary'].replace(0, np.nan)
df_copy['job_offer_max_salary'] = df_copy['job_offer_max_salary'].fillna(median_salary)

In [23]:
df_copy.loc[df_copy['job_offer_max_salary'] == df_copy['job_offer_max_salary'].min(),:]

,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
92,279618,AIエンジニア募集(関東),"['東京都', '神奈川県', 'フルリモート']",35,80.0,"['Python', 'AWS', 'TensorFlow', 'Azure', 'GCP'...",株式会社 テックエデュケイションカンパニー,150,2013-02-28 15:00:00,AIエンジニア


In [24]:
df_copy['job_tag'].value_counts()

job_tag
その他            174
AIエンジニア        108
データエンジニア        84
データサイエンティスト     73
Name: count, dtype: int64